In [10]:
import numpy as np
import pandas as pd
import geopandas as gpd
import branca.colormap as cm
import folium
import fiona
import shapefile
import rasterstats
import math
import json

from folium.plugins import MarkerCluster
from folium.features import ColorLine
from folium.features import GeoJson
from shapely.geometry import mapping
from shapely.geometry import LineString
from shapely.geometry import Polygon, mapping
from rasterio.mask import mask
from geopy.distance import geodesic

In [11]:
def distance_measure(route_shp):
    lines_gdf = extract_point_df(route_shp)
    dis = []
    for idx in range(len(lines_gdf)-1):
        coordinate_1 = lines_gdf.loc[idx]['coordinates']
        coordinate_2 = lines_gdf.loc[idx + 1]['coordinates']
        swap_coord_1= (coordinate_1[1], coordinate_1[0])
        swap_coord_2= (coordinate_2[1], coordinate_2[0])
        dis.append(geodesic(swap_coord_1,swap_coord_2).m)

    dist = np.cumsum(dis)
    
    return(dis, dist)


In [12]:
def gradient(route_shp, route_num, rasterfile):
    elevation = rasterstats.point_query(route_shp, rasterfile)
    route_distance,_ = distance_measure(route_shp)
    route_gradient =  np.insert(abs(np.diff(elevation)/ route_distance),0, 0)
    
    return route_gradient        

In [13]:
def make_lines(gdf, gradient, idx, geometry = 'geometry'):   
    coordinate_1 = gdf.loc[idx]['coordinates']
    coordinate_2 = gdf.loc[idx + 1]['coordinates']
    line = LineString([coordinate_1, coordinate_2])
    data = {'gradient': gradient,
            'geometry':[line]}
    df_line = pd.DataFrame(data, columns = ['gradient', 'geometry'])
    
    return df_line

In [14]:
def make_multi_lines(linestring_route_df, elevation_gradient):
    df_route = pd.DataFrame(columns = ['gradient', 'geometry'])
    for idx in range(len(linestring_route_df) - 1):
        df_linestring = make_lines(linestring_route_df, elevation_gradient[idx], idx)
        df_route = pd.concat([df_route, df_linestring])
    gdf_route = gpd.GeoDataFrame(df_route)
    return gdf_route

In [15]:
def read_shape(shapefile, route_num):
    routes_shp= gpd.read_file(shapefile)
    route_shp = routes_shp[routes_shp['ROUTE_NUM'] == route_num]
    return route_shp

In [16]:
def extract_point_df(route_shp):
    route_geometry = route_shp.geometry.values # list of shapely geometries
    route_geometry = [mapping(route_geometry[0])]
    coordinates_route = route_geometry[0]['coordinates']
    linestring_route = []
    for i in range(len(coordinates_route)):
        linestring_route.append(coordinates_route[i][:2])
        linestring_route_df = pd.DataFrame()  
        linestring_route_df['coordinates'] = linestring_route
    return linestring_route_df

In [17]:
def route_map(gdf_route):
    """Visualize route_num map according to gradient. 
    e.g. route_num = 45 , shows plot of route_45."""
    UW_coords = [47.655548, -122.303200]
    figure_size = folium.Figure(height = 400)
    route_map = folium.Map(location = UW_coords, zoom_start = 12)
    min_grade = min(gdf_route['gradient'])
    max_grade = max(gdf_route['gradient'])
    route_json = gdf_route.to_json()
    linear_map = cm.linear.RdYlBu_04.scale(min_grade, max_grade )
    route_layer = folium.GeoJson(route_json, style_function = lambda feature: {
        'color': linear_map(feature['properties']['gradient']),
        'weight': 8})
    route_layer.add_child
    route_map.add_child(linear_map)
    route_map.add_child(route_layer)
    route_map.add_to(figure_size)
    return route_map

In [21]:
def route_to_shp(route_num, shapefile, rasterfile):
    """input the number of route, then output a GeoDataFrame with gradient and geometry
    information of that route, and elevation_gradient for each line segment. 
    Also will save the route as shapefile named 'route_num'."""
    route_shp = read_shape(shapefile, route_num)
    
    linestring_route_df = extract_point_df(route_shp)
    
    elevation_gradient = gradient(route_shp, route_num, rasterfile)

    gdf_route = make_multi_lines( linestring_route_df, elevation_gradient)  
    
    return route_map(gdf_route)

In [22]:
route_to_shp(45,'../data/six_routes.shp','../data/sea_dtm_north.tif')